# Study of the symmetry axis

In [ ]:
from shapeOptInductor import create_plots, update_plots, rot
import ngsolve as ngs
from ngsolve.webgui import Draw
import numpy as np
import matplotlib.pyplot as plt 
from netgen.geom2d import SplineGeometry

## 1 - Meshing of the inductor

In [ ]:
from netgen.geom2d import SplineGeometry
from debug import *

def gen_mesh(air_gap=4.11e-3, maxh=1e-3, debug=False):
    """Gives a triangular mesh"""
    # Global domain
    r = 0.04

    # Geometry definition
    a = 1e-2
    h = 25e-3
    ba = 1e-2
    geo = SplineGeometry()
    pnts = [
        (0, 0),  # p0
        (a / 2, 0),  # p1
        (a / 2 + ba, 0),  # p2
        (r, 0),  # p3
        (r, r),  # p4
        (0, r),  # p5
        (0, h/2),  # p6
        (0, air_gap / 2),  # p7
        (a / 2, air_gap / 2),  # p8
        (a / 2, h / 2 - a / 2),  # p9
        (a / 2 + ba, h / 2 - a / 2),  # p10
        (a / 2 + ba, air_gap / 2),  # p11
        (a + ba, air_gap / 2),  # p12
        (a + ba, h / 2),  # p13
        ]
    
    maxhFine = maxh / 10
    maxhFine = maxh
    maxhMed = maxh / 5
    maxhMed = maxh 
    pointH = [
        maxh,  # p0
        maxh,  # p1
        maxh,  # p2
        maxh,  # p3
        maxh,  # p4
        maxh,  # p5
        maxh,  # p6
        maxhFine,  # p7
        maxhFine,  # p8
        maxh,  # p9
        maxh,  # p10
        maxhFine,  # p11
        maxhFine,  # p12
        maxh,  # p13
    ]

    (
        p0,
        p1,
        p2,
        p3,
        p4,
        p5,
        p6,
        p7,
        p8,
        p9,
        p10,
        p11,
        p12,
        p13,
    ) =  [
        geo.AppendPoint(*pnts[i], pointH[i]) for i in range(len(pnts))
    ]

    # List of lines with boundary conditions and domains
    lines = [
        [["line", p0, p1], {"bc": "a", "leftdomain": 3, "rightdomain": 0, "maxh": maxhMed}],
        [["line", p1, p2], {"bc": "b", "leftdomain": 2, "rightdomain": 0, "maxh": maxhMed}],
        [["line", p2, p3], {"bc": "c", "leftdomain": 4, "rightdomain": 0}],
        [["spline3", p3, p4, p5], {"bc": "d", "leftdomain": 4, "rightdomain": 0}],
        [["line", p5, p6], {"bc": "e", "leftdomain": 4, "rightdomain": 0}],
        [["line", p6, p7], {"bc": "f", "leftdomain": 1, "rightdomain": 0, "maxh": maxhMed}],
        [["line", p7, p0], {"bc": "g", "leftdomain": 3, "rightdomain": 0, "maxh": maxhMed}],
        [["line", p7, p8], {"bc": "h", "leftdomain": 1, "rightdomain":3, "maxh": maxhFine}],
        [["line", p8, p9], {"bc": "i", "leftdomain": 1, "rightdomain":2, "maxh": maxhMed }],
        [["line", p9, p10], {"bc": "j", "leftdomain": 1, "rightdomain":2, "maxh": maxhMed }],
        [["line", p10, p11], {"bc": "k", "leftdomain": 1, "rightdomain":2, "maxh": maxhMed }],
        [["line", p11, p12], {"bc": "l", "leftdomain": 1, "rightdomain": 4, "maxh": maxhFine}],
        [["line", p12, p13], {"bc": "m", "leftdomain": 1, "rightdomain": 4, "maxh": maxhMed}],
        [["line", p13, p6], {"bc": "n", "leftdomain": 1, "rightdomain": 4, "maxh": maxhMed}],
        [["line", p1, p8], {"bc": "o", "leftdomain": 3, "rightdomain": 2, "maxh": maxhMed}],
        [["line", p2, p11], {"bc": "p", "leftdomain": 2, "rightdomain": 4, "maxh": maxhMed}],
        
    ]

    # Append all lines to the geometry
    for line, props in lines:
        geo.Append(line, **props)

    # Optional matpotlib debug render
    if debug : 
        fig, axes = plt.subplots(2, 2, figsize=(15, 15))
        debug_geo(geo, axes[0, 0])
        debug_points(geo, 
                     # point_names is optional
                     point_names="p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13".split(","),
                     ax=axes[1, 0]
                     )
        debug_bc_names(geo, ax=axes[0, 1])
        debug_region_labels(geo, ax=axes[1, 1])
        plt.show()

    # Set materials and meshing parameters
    geo.SetMaterial(1, "core")
    geo.SetMaterial(2, "coil")
    geo.SetMaterial(3, "air")
    geo.SetMaterial(4, "air")
    ngmesh = geo.GenerateMesh(maxh=maxh)
    # new_ngmesh = ngmesh.Mirror((0, 0, 0), (-1, 0, 0)).Mirror((0, 0, 0), (0, -1, 0))
    # coilL_id = new_ngmesh.AddRegion("coilL", 2)
    return ngmesh

maxh = 7e-4
ngmesh = gen_mesh(maxh=maxh, debug=False)
mesh0 = ngs.Mesh(ngmesh) # Partial mesh 
mesh = ngs.Mesh(ngmesh.Mirror((0, 0, 0), (-1, 0, 0)).Mirror((0, 0, 0), (0, -1, 0))) # Full mesh 

XiAir = mesh.MaterialCF({"air": 1})
XiCore = mesh.MaterialCF({"core": 1})
XiCoil = mesh.MaterialCF({"coil": 1})
XiCoilR = ngs.IfPos(ngs.x, XiCoil, 0)
XiCoilL = ngs.IfPos(ngs.x, 0, XiCoil)

# Draw(1 * XiAir + 2 * XiCoil + 3 * XiCore, mesh, radius=0.02)
# Draw(1 * XiAir + 2 * XiCoil + 3 * XiCore, mesh0, radius=0.02)
Draw(XiCoilR - XiCoilL, mesh, radius=0.02)
Draw(XiCoilR - XiCoilL, mesh0, radius=0.02)

## 2 - Computation of magnetic state and quantities of interest 

In [ ]:
# Frequency
f = 5e4  # Hz
omega = 2 * np.pi * f

# Magnetic
mu0 = 4e-7 * np.pi
mur = 1000
mu_iron = mur * mu0
delta = 0.1
mu_coil = np.exp( -1j * delta) * mu0  #  AC losses in the copper from the imaginary part of the permeability

# Current
nb_turn = 200  # Number of turn in the coil
Is = 2  # Source current intensity
js = nb_turn * Is / (ngs.Integrate(XiCoilR, mesh)) * (XiCoilR - XiCoilL)  # Source current density
thickness = 1e-2
coeff_losses = omega / 2 * thickness * (1 / mu_coil).imag

def magWeakFormComplex(a, a_):
    bf = ngs.grad(a_) * 1 / mu_iron * ngs.grad(a) * ngs.dx("core")
    bf += ngs.grad(a_) * 1 / mu_coil * ngs.grad(a) * ngs.dx("coil")
    bf += ngs.grad(a_) * 1 / mu0 * ngs.grad(a) * ngs.dx("air")
    lf = a_ * js * ngs.dx("coil")
    return bf, lf


def solveStateComplex(fes):
    a, a_ = fes.TnT()
    bf, f = magWeakFormComplex(a, a_)
    K, F = ngs.BilinearForm(fes), ngs.LinearForm(fes)
    K += bf
    F += f
    K.Assemble()
    F.Assemble()
    gf = ngs.GridFunction(fes)
    Kinv = K.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")  # Hermitian ?
    gf.vec.data = Kinv * F.vec
    return gf, Kinv


fes = ngs.H1(mesh, order=1, dirichlet="d", complex=True)
state, Kinv = solveStateComplex(fes)
Draw(state.real, mesh, radius=0.02)

fes0 = ngs.H1(mesh0, order=1, dirichlet="d|e|f|g", complex=True)
state0, Kinv0 = solveStateComplex(fes0)
Draw(state0.real, mesh0, radius=0.02)

In [ ]:
def Inductance(a, mesh, lz=1):
    rel = XiAir / mu0 + XiCoil / mu_coil + XiCore / mu_iron
    return 2 * lz * thickness / (2 * Is**2) * ngs.Integrate(rel.real * ngs.Norm(ngs.grad(a)) ** 2, mesh)


def Losses(a, mesh, lz=1):
    return ngs.Integrate(lz * coeff_losses * XiCoil * ngs.Norm(ngs.grad(a)) ** 2, mesh)


In [ ]:
print("Full geometry:")
print(f"The inductance is {1e3 * np.absolute(Inductance(state, mesh)):.3f} mH")
print(f"The losses amounts to {Losses(state, mesh):.3f} W")
print("\n")
print("Partial geometry:")
print(f"The inductance is {1e3 * np.absolute(Inductance(state0, mesh0, lz=4)):.3f} mH")
print(f"The losses amounts to {Losses(state0, mesh0, lz=4):.3f} W")

## 3 - Computation of adjoint states and shape derivatives

In [ ]:
def solveAdjointLosses(a0, Kinv):
    """Solves the adjoint equation for the losses inside the coil"""
    fes = a0.space
    p, p_ = fes.TnT()
    F = ngs.LinearForm(fes)
    F += - 2 * coeff_losses * ngs.InnerProduct(ngs.grad(a0), ngs.grad(p_)) * ngs.dx("coil")
    F.Assemble()
    gf = ngs.GridFunction(fes)
    gf.vec.data = Kinv.H * F.vec
    return gf


def solveAdjointInductance(a0, Kinv):
    """Solves the adjoint equation for the inductance"""
    fes = a0.space
    p, p_ = fes.TnT()
    F = ngs.LinearForm(fes)
    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    coeff_induc =  2 * thickness / (2 * Is**2) * rel
    F += -2 * coeff_induc * ngs.InnerProduct(ngs.grad(a0), ngs.grad(p_)).real * ngs.dx
    F.Assemble()
    gf = ngs.GridFunction(fes)
    gf.vec.data =  Kinv * F.vec
    return gf


def computeLossesShapeDerivative(mesh, VEC_complex, VEC_real, a0, p0):
    """Shape derivative for the energy-based losses inside the coil"""
    X = VEC_complex.TestFunction()

    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    Id = ngs.CoefficientFunction((1, 0, 0, 1), dims=(2, 2))  # Identity matrix
    dA = ngs.div(X) * Id - ngs.grad(X) - ngs.grad(X).trans

    dLOmega = ngs.LinearForm(VEC_complex)
    dLOmega += coeff_losses * XiCoil * ngs.InnerProduct(dA * ngs.grad(a0), ngs.grad(a0)) * ngs.dx("coil")
    dLOmega += -js * XiCoil * ngs.div(X) * p0 * ngs.dx("coil")
    dLOmega += ngs.InnerProduct(rel * dA * ngs.grad(a0), ngs.grad(p0)) * ngs.dx("coil")
    dLOmega += ngs.InnerProduct(rel * dA * ngs.grad(a0), ngs.grad(p0)) * ngs.dx("air|core")

    dLOmega.Assemble()
    dJ = ngs.GridFunction(VEC_real)
    dJ.vec.FV().NumPy()[:] = np.real(dLOmega.vec.FV().NumPy()[:])
    return dJ


def computeInductanceShapeDerivative(mesh, VEC_complex, VEC_real, a0, p0):
    """Shape derivative for the inductance"""
    X = VEC_complex.TestFunction()

    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    Id = ngs.CoefficientFunction((1, 0, 0, 1), dims=(2, 2))  # Identity matrix
    coeff_induc = 2 * thickness / (2 * Is**2) * rel
    dA = ngs.div(X) * Id - ngs.grad(X) - ngs.grad(X).trans

    dLOmega = ngs.LinearForm(VEC_complex)
    dLOmega += coeff_induc * ngs.InnerProduct(dA * ngs.grad(a0), ngs.grad(a0)) * ngs.dx
    dLOmega += -js * ngs.div(X) * p0 * ngs.dx
    dLOmega += ngs.InnerProduct(rel * dA * ngs.grad(a0), ngs.grad(p0)) * ngs.dx
    dLOmega.Assemble()

    dJ = ngs.GridFunction(VEC_real)
    dJ.vec.FV().NumPy()[:] = np.real(dLOmega.vec.FV().NumPy()[:])
    return dJ

In [ ]:
print("Full geometry:")
VEC_complex = ngs.VectorH1(mesh, complex=True)
VEC_real = ngs.VectorH1(mesh)

state, Kinv = solveStateComplex(fes)
adjoint_losses = solveAdjointLosses(state, Kinv)
adjoint_inductance = solveAdjointInductance(state, Kinv)

dJOmega = computeInductanceShapeDerivative(mesh, VEC_complex, VEC_real, state, adjoint_inductance)
# dJOmega = computeLossesShapeDerivative(mesh, VEC_complex, VEC_real, state, adjoint_losses)
Draw(adjoint_inductance, mesh, radius=0.02)
Draw(adjoint_losses.real, mesh, radius=0.02)  # Note that the imag part should be 0

print("Partial geometry:")
VEC_complex0 = ngs.VectorH1(mesh0, complex=True)
VEC_real0 = ngs.VectorH1(mesh0)

state0, Kinv0 = solveStateComplex(fes0)
adjoint_losses0 = solveAdjointLosses(state0, Kinv0)
adjoint_inductance0 = solveAdjointInductance(state0, Kinv0)

dJOmega0 = computeInductanceShapeDerivative(mesh0, VEC_complex0, VEC_real0, state0, adjoint_inductance0)
# dJOmega0 = computeLossesShapeDerivative(mesh0, VEC_complex0, VEC_real0, state0, adjoint_losses0)
Draw(adjoint_inductance0, mesh0, radius=0.02)
Draw(adjoint_losses0.real, mesh0, radius=0.02)  # Note that the imag part should be 0

In [ ]:
def SolveDeformationEquation(mesh, fX, dirichlet="", dirichletx="", dirichlety="", definedon="air|core|coil"):
    VEC = ngs.VectorH1(
        mesh,
        order=1,
        dirichlet=dirichlet,
        dirichletx=dirichletx,
        dirichlety=dirichlety,
        definedon=definedon,
    )
    PHI, X = VEC.TnT()

    B = ngs.BilinearForm(VEC)
    B += ngs.InnerProduct(X, PHI) * ngs.dx
    B.Assemble()

    C = ngs.LinearForm(VEC)
    C += ngs.InnerProduct(fX, X) * ngs.dx
    C.Assemble()

    gfX = ngs.GridFunction(VEC)
    gfX.vec.data = B.mat.Inverse(VEC.FreeDofs()) * C.vec
    return gfX

print("Shape derivative on full geometry :")
dJ = ngs.GridFunction(VEC_real)
dJ.vec.FV().NumPy()[:] = dJOmega.vec.FV().NumPy()[:]
Draw(dJ, mesh, vectors={"grid_size": 40}, radius=0.02)

print("Shape derivative on partial geometry :")
print("On the symmetry axes, components that should cancels because of the symmetry of the field does not cancels naturally.")
print("This is due to missing symmetrical nodal elements during the assembly.")
dJ0 = ngs.GridFunction(VEC_real0)
dJ0.vec.FV().NumPy()[:] = dJOmega0.vec.FV().NumPy()[:]
Draw(dJ0.real, mesh, vectors={"grid_size": 40}, radius=0.02)

print("Shape derivative on partial geometry after manual symmetry correction :")
gf1 = ngs.GridFunction(VEC_real0)
gf1.Set((-dJ0.components[0], dJ0.components[1]), definedon=mesh.Boundaries("e|f|g"))
gf2 = ngs.GridFunction(VEC_real0)
gf2.Set((dJ0.components[0], -dJ0.components[1]), definedon=mesh.Boundaries("a|b|c"))
gf3 = ngs.GridFunction(VEC_real0)
gf3.vec.data[0] = -dJ0.components[0].vec.data[0]
gf3.vec.data[VEC_real0.ndof//2] = -dJ0.components[1].vec.data[0]
gf = ngs.GridFunction(VEC_real0)
gf.vec.data = dJ0.vec + gf1.vec + gf2.vec + gf3.vec
Draw(gf, mesh0, vectors={"grid_size": 80})

print("The difference is around numerical precision, this is ok :")
Draw(dJ - gf, mesh0, vectors={"grid_size": 80})

print("Shape derivative on partial geometry after regularization-type symmetry correction :")
print("It does not work that well...")
gf4 = SolveDeformationEquation(mesh0, dJ0, dirichletx="e|f|g", dirichlety="a|b|c")
Draw(gf4.real, mesh0, vectors={"grid_size": 40}, radius=0.02)

print("The difference is significant :")
Draw(dJ - gf4, mesh0, vectors={"grid_size": 80})